In [1]:
from lxml import etree
import pandas as pd
from io import StringIO
import requests
from urllib.parse import quote_plus

In [2]:
# url = 'https://www.fpds.gov/ezsearch/fpdsportal?s=FPDS&indexName=awardfull&templateName=1.5.2&q=LAST_MOD_DATE%3A%5B2020%2F04%2F02%2C2020%2F04%2F02%5D&rss=1&feed=atom0.3'
# url = 'https://www.fpds.gov/ezsearch/fpdsportal?s=FPDS.GOV&indexName=icdindex&templateName=1.5.2&q=+DEPARTMENT_NAME%3A%22General+Services+Administration%22&rss=1&feed=atom0.3'
# url = 'https://www.fpds.gov/ezsearch/search.do?indexName=awardfull&templateName=1.5.2&s=FPDS.GOV&q=financial+assistance+actions&rss=1&feed=atom0.3'
# url = 'https://www.fpds.gov/ezsearch/fpdsportal?s=FPDS.GOV&indexName=awardfull&templateName=1.5.2&rss=1&feed=atom0.3&q=+LAST_MOD_DATE%3A%5B2021%2F10%2F01%2C2021%2F11%2F01%5D'

base_url = 'https://www.fpds.gov/ezsearch/fpdsportal?s=FPDS.GOV&indexName=awardfull&templateName=1.5.2&rss=1&feed=atom0.3&q=+'
start_date = '2021/11/28'
end_date = '2021/11/28'
param = f'LAST_MOD_DATE:[{start_date},{end_date}]'

url = base_url + quote_plus(param)

print(url)


https://www.fpds.gov/ezsearch/fpdsportal?s=FPDS.GOV&indexName=awardfull&templateName=1.5.2&rss=1&feed=atom0.3&q=+LAST_MOD_DATE%3A%5B2021%2F11%2F28%2C2021%2F11%2F28%5D


In [ ]:
response = requests.get(url)
print(response.ok)

In [ ]:
doc = etree.XML(response.content)

In [ ]:
xslt_tree = etree.parse("fpds_feed.xsl")
transform = etree.XSLT(xslt_tree)

In [ ]:
result = transform(doc)

In [ ]:
csv_str =str(result)
# csv_str

In [ ]:
mystr = StringIO(csv_str)

df = pd.read_csv(mystr, sep="\t", header=None, index_col=0)
df.head(10)

In [ ]:
df_new = df.transpose(copy=True)
df_new.iloc[:, :10]

In [ ]:
df_new.shape

In [ ]:
# output dataframe to a .csv file
df_new.to_csv("fpds_feed_result.csv", index=False)

# Get count of records returned

In [ ]:
f = StringIO(response.text)
tree = etree.XML(response.content)
href = tree.xpath("/f:feed/f:link[@rel = 'last']/@href", namespaces={'f': 'http://www.w3.org/2005/Atom'})[0]
print(href)

In [ ]:
import re

x = re.search("start=(\d+)$", href)
print(x.group(1))


# Test fpds_feed_requests

In [ ]:
from fpds_feed_requests import requests

In [ ]:
req = requests(base_url)
req.get(param, start_index=0, max_workers = 2)

# Test fpds_feed_reader

In [3]:
from fpds_feed_reader import feed_reader

In [4]:
reader = feed_reader()


In [5]:
reader.get_data(param)

Requesting data (~18180 rows) from the feed...


TypeError: decorator() got an unexpected keyword argument 'max_workers'

# Test Multi-threaded Requests 

In [ ]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from time import time

url_list = [(start, url + f"&start={start}") for start in range(0, 100, 10)]
# print(url_list)

def download_file(url, index):
    response = requests.get(url, stream=True)
    # return response.content
    return index

start = time()

processes = []
with ThreadPoolExecutor(max_workers=10) as executor:
    for url in url_list:
        processes.append(executor.submit(download_file, url[1], url[0]))

for task in as_completed(processes):
    print(task.result())


print(f'Time taken: {time() - start}')

In [ ]:
from fpds_feed_requests import requests as t_requests
from xml_transform import transform

In [ ]:
t = transform("fpds_feed.xsl")
req = t_requests(base_url)
iter = req.get(param, start_index=42200, max_workers=10)
results = []
df = None
for task in iter:
    # results.append(task.result())
    df = t.to_dataframe(task.result())

df.index

In [ ]:
from fpds_feed_reader import feed_reader
from datetime import date, timedelta

In [ ]:
reader = feed_reader()
created_date = date(1979, 1 , 15)
end_date = date.today()

while created_date <= end_date:
    cd = created_date.strftime("%Y/%m/%d")
    print(cd)
    param = f"CREATED_DATE:[{cd},{cd}]"
    reader.get_data(param)
    created_date += timedelta(days=1)

In [ ]:


my_date = "2021-12-01"
datetime.datetime.strptime(my_date, "%Y-%m-%d")



In [1]:
def my_func(x,y):
    return x+y

print(type(my_func))


<class 'function'>
